In [1]:
import json
import glob
import pandas as pd
import numpy as np

def getDF(path):
    ## json load
    data = json.loads(open(path, 'r', encoding='utf-8-sig').read())['rows']
    
    if len(data) < 1:
        print(f'>>> json File : \"{path}\" is Data Not Found...')
        return None;
    
    df = pd.DataFrame()
    # json 구조에서 경기정보 dataframe 변환
    for d in data:
        tmp = pd.DataFrame(d['row'])['Text']
        if len(d['row'])==8:
            tmp[8] = np.NaN
            tmp = tmp.shift(1)

        df = df.append(tmp, ignore_index=True)
    
    ########### 전처리
    # 필요없는 문자열 삭제
    df.replace('<b>', '', regex=True, inplace=True)
    df.replace('</b>', '', regex=True, inplace=True)
#     df.replace('<span>', '', regex=True, inplace=True)
#     df.replace('</span>', '', regex=True, inplace=True)
#     df.replace('<em>', '', regex=True, inplace=True)
#     df.replace('</em>', '', regex=True, inplace=True)
    df.replace('<span class="win">', '*', regex=True, inplace=True)
    df.replace('<span class="lose">', '', regex=True, inplace=True)

    df[0] = df[0].fillna(method='ffill').apply(lambda x: x if x.find('(') == -1 else x[:x.find('(')]) # 요일 삭제

    del df[3] # 리뷰링크 삭제
    del df[4] # 하이라이트 링크 삭제
    del df[5] # TV 삭제
    del df[6] # 라디오 삭제

    df[9] = np.NaN
    df = df.shift(1, axis=1)
    df[0] = path[:-8].split('_')[-1:][0] # 파일명에서 연도를 추가
    
    print(f'>>> json File : \"{path}\" is Load Complete!!!')
    return df

## 연도 기간을 입력하면 해당되는 파일 목록이 출력
# getAsosFilePath('./tmp/', ['2018', '2019'])
# './teamRawData_json/data/0,9_2018_05.json'
def getjsonFilePath(parDirPath, yearlist):
    result = []
    yearlist = set(yearlist) # 연도 목록 중복제거
    monList = ['01','02','03','04','05','06','07','08','09','10','11','12']
    for y in yearlist:
        for m in monList:
            result.append(f'{parDirPath}0,9_{y}_{m}.json')    
    return result
    

In [199]:
dirPath = './teamRawData_json/data/'
# fileList = glob.glob(dirPath + '*')
# fileList.sort()

li = ['2018','2019','2020','2021']
pathList = getjsonFilePath(dirPath, li) # 경로 생성
pathList.sort()
df = pd.DataFrame()
for path in pathList:
    tmp = getDF(path)
    if type(tmp) != type(None):
        df = df.append(tmp, ignore_index=True)


        
        
## 연월일 시간 합체. dtype datetime으로 변경
df[0] = df[0] + '.' + df[1] + ' ' + df[2]
del df[1]
del df[2]
df[0] = pd.to_datetime(df[0])

## 경기정보 컬럼 html tag 제거
## 우승팀은 * 표시
# 반드시 한번만 실행해야됨.
df[7] = df[7].apply(lambda x:x.replace('<span class="same">','_').replace('<span>','_').replace('</span>','_').replace('<em>','_').replace('</em>','_').replace('__','_').replace('__','_')[1:-1])

## 
## tmp[9].unique()
## >> array(['-', '우천취소', '미세먼지취소', '그라운드사정', '강풍취소', '기타'], dtype=object)
df.drop(df[df[9] != '-'].index, inplace=True) # 경기취소 304 row 삭제

df['home'] = df[7].apply(lambda x : x.split('_vs_')[1])
df['away'] = df[7].apply(lambda x : x.split('_vs_')[0])
del df[7]

df.rename(columns={0:'date'}, inplace=True)
df.rename(columns={8:'park'}, inplace=True)
df.rename(columns={9:'asos'}, inplace=True)

# 서로 인접한 지역(고척, 잠실)은 동일한 asos 관측소를 사용한다.
# 가장 인접한 관악산 116 관측소는 1990년 폐쇄되었으므로 국립기상박물관 108 관측소를 기준으로 한다.
park2asos = {
    '잠실': '108',
    '고척': '108',
    '문학': '112',
    '수원': '119',
    '청주': '131',
    '대전': '133',
    '포항': '138',
    '대구': '143',
    '울산': '152',
    '마산': '155',
    '창원': '155',
    '광주': '156',
    '사직': '159'
}

df['asos'] = df['park'].apply(lambda x: park2asos[x])
df['homeWin'] = np.NaN
df['home_score'] = df['home'].apply(lambda x: x.split('_')[0].replace('*',''))
df['away_score'] = df['away'].apply(lambda x: x.split('_')[1].replace('*',''))

df['home'] = df['home'].apply(lambda x: x.split('_')[1])
df['away'] = df['away'].apply(lambda x: x.split('_')[0])

## 2018-07 올스타전 삭제(드림-나눔). 2건
df.drop(df[df['home'] == '드림'].index, inplace=True)
df.drop(df[df['away'] == '드림'].index, inplace=True)

name2Nick = {
    'KIA': '기아',
    'KT': 'KT',
    'LG': 'LG',
    'NC': 'NC',
    'SK': 'SK',
    'SSG': 'SK',
    '넥센': '키움',
    '두산': '두산',
    '롯데': '롯데',
    '삼성': '삼성',
    '키움': '키움',
    '한화': '한화'
}
df['home'] = df['home'].apply(lambda x: name2Nick[x])
df['away'] = df['away'].apply(lambda x: name2Nick[x])

df['home_score'] = pd.to_numeric(df['home_score'])
df['away_score'] = pd.to_numeric(df['away_score'])

df.reset_index(drop=True, inplace=True)
df.homeWin = df.home_score - df.away_score

df

>>> json File : "./teamRawData_json/data/0,9_2018_01.json" is Data Not Found...
>>> json File : "./teamRawData_json/data/0,9_2018_02.json" is Data Not Found...
>>> json File : "./teamRawData_json/data/0,9_2018_03.json" is Load Complete!!!
>>> json File : "./teamRawData_json/data/0,9_2018_04.json" is Load Complete!!!
>>> json File : "./teamRawData_json/data/0,9_2018_05.json" is Load Complete!!!
>>> json File : "./teamRawData_json/data/0,9_2018_06.json" is Load Complete!!!
>>> json File : "./teamRawData_json/data/0,9_2018_07.json" is Load Complete!!!
>>> json File : "./teamRawData_json/data/0,9_2018_08.json" is Load Complete!!!
>>> json File : "./teamRawData_json/data/0,9_2018_09.json" is Load Complete!!!
>>> json File : "./teamRawData_json/data/0,9_2018_10.json" is Load Complete!!!
>>> json File : "./teamRawData_json/data/0,9_2018_11.json" is Data Not Found...
>>> json File : "./teamRawData_json/data/0,9_2018_12.json" is Data Not Found...
>>> json File : "./teamRawData_json/data/0,9_201

,date,park,asos,home,away,homeWin,home_score,away_score
0,2018-03-24 14:00:00,마산,155,NC,LG,2,4,2
1,2018-03-24 14:00:00,잠실,108,두산,삼성,-3,3,6
2,2018-03-24 14:00:00,문학,112,SK,롯데,1,6,5
3,2018-03-24 14:00:00,광주,156,기아,KT,-1,4,5
4,2018-03-24 14:00:00,고척,108,키움,한화,3,6,3
...,...,...,...,...,...,...,...,...
2876,2021-10-30 17:00:00,사직,159,롯데,LG,2,4,2
2877,2021-10-30 17:00:00,창원,155,NC,삼성,-6,5,11
2878,2021-10-30 17:00:00,광주,156,기아,키움,-5,1,6
2879,2021-10-30 17:00:00,대전,133,한화,두산,-2,3,5


In [202]:
df.to_excel("teamRawData_Fromjson.xlsx", index=False)

In [427]:
idx = ['date', 'park', 'asosCode', 'home', 'away', 'win']

2883